In [ ]:
# @title Paso 1: Configuración Completa
from google.colab import drive
from pathlib import Path
import os
import requests
import time

# --- Configuración Interactiva ---
version = "1.21.1"       #@param ["1.21.1", "1.20.4", "1.19.4"]
ram = "12G"              #@param ["6G", "8G", "10G", "12G"]
server_type = "paper"    #@param ["paper", "vanilla", "fabric"]
tunnel_type = "ngrok"    #@param ["ngrok", "playit"]

# --- Inicialización ---
print("🛠️ Iniciando configuración del servidor...")
drive.mount('/content/drive', force_remount=True)
server_path = Path("/content/drive/My Drive/Minecraft-Server")
server_path.mkdir(parents=True, exist_ok=True)
os.chdir(server_path)

# --- Instalación de Java ---
java_version = "17" if any(v in version for v in ["1.19", "1.18"]) else "21"
print(f"☕ Instalando Java {java_version}...")
!sudo apt update -qq && sudo apt install -y openjdk-{java_version}-jre-headless

# --- Configuración del Túnel ---
if tunnel_type == "ngrok":
    print("\n🔌 Configurando Ngrok...")
    !curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null
    !echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list
    !sudo apt update && sudo apt install -y ngrok

    print("\n🔑 Requerimiento de Token:")
    print("1. Obtén tu token gratis en: https://dashboard.ngrok.com/get-started/your-authtoken")
    ngrok_token = input("2. Ingresa tu token de Ngrok: ").strip()
    !ngrok config add-authtoken $ngrok_token
else:
    print("\n🎮 Configurando Playit.gg...")
    !curl -sSL https://github.com/playit-cloud/playit-agent/releases/latest/download/playit-linux-amd64 -o playit
    !chmod +x playit && sudo mv playit /usr/local/bin/
    (server_path / "playit-config").mkdir(exist_ok=True)
    !ln -sf "$(pwd)/playit-config" "/root/.config/playit"

# --- Descarga del Servidor ---
print(f"\n⬇️ Descargando {server_type} {version}...")
if server_type == "paper":
    api_url = f"https://api.papermc.io/v2/projects/paper/versions/{version}"
    build = requests.get(api_url).json()["builds"][-1]
    !wget -q --show-progress -O server.jar "{api_url}/builds/{build}/downloads/paper-{version}-{build}.jar"
elif server_type == "fabric":
    !wget -q --show-progress -O server.jar "https://meta.fabricmc.net/v2/versions/loader/{version}/latest/server/jar"
else:
    manifest = requests.get("https://launchermeta.mojang.com/mc/game/version_manifest.json").json()
    version_data = next(v for v in manifest["versions"] if v["id"] == version)
    jar_url = requests.get(version_data["url"]).json()["downloads"]["server"]["url"]
    !wget -q --show-progress -O server.jar "$jar_url"

# --- Configuración Final ---
!echo "eula=true" > eula.txt
(server_path / "plugins").mkdir(exist_ok=True)

print("\n✅ ¡Configuración completada!")
print(f"""
📋 Resumen:
- Versión: {version}
- RAM: {ram}
- Tipo: {server_type}
- Túnel: {tunnel_type}
- Ruta: {server_path}
""")
print("➡️ Ejecuta el 'Paso 2: Inicio del Servidor' para comenzar")

In [ ]:
# %% [markdown]
## 🚀 **Paso 2: Inicio del Servidor**

# @title Paso 2: Inicio del Servidor
from google.colab import drive
from pathlib import Path
import os
import time
import subprocess
import requests

# --- Configuración Interactiva ---
tunnel_type = "playit" #@param ["ngrok", "playit"]

# --- Inicialización ---
print("🚀 Iniciando el servidor...")
drive.mount('/content/drive', force_remount=False)
server_path = Path("/content/drive/My Drive/Minecraft-Server")
os.chdir(server_path)

# --- Cargar configuración del Paso 1 ---
config_file = server_path / "server_config.txt"
config = {}
try:
    with open(config_file, "r") as f:
        for line in f:
            if "=" in line:
                key, value = line.strip().split("=", 1)
                config[key] = value
    version = config.get("version", "1.21.1")
    ram = config.get("ram", "6G")
except FileNotFoundError:
    print("⚠️ No se encontró el archivo de configuración. Asegúrate de haber ejecutado el Paso 1.")
    exit()

# --- Verificar e instalar Java si es necesario ---
def check_java(version):
    try:
        java_check = subprocess.run(['java', '-version'], capture_output=True, text=True, check=True)
        return f"openjdk version \"{version}\"" in java_check.stderr or f"Java(TM) SE Runtime Environment (build {version}" in java_check.stderr
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

java_major_version = "17" if any(v in version for v in ["1.19", "1.18"]) else "21"
if not check_java(java_major_version):
    print(f"☕ Instalando Java {java_major_version}...")
    !sudo apt update -qq
    !sudo apt install -y openjdk-{java_major_version}-jre-headless
else:
    print(f"☕ Java {java_major_version} ya está instalado.")

# --- Verificar e instalar Ngrok si es necesario ---
def check_ngrok():
    try:
        subprocess.run(['ngrok', 'version'], check=True, capture_output=True)
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

if tunnel_type == "ngrok":
    if not check_ngrok():
        print("\n🔌 Instalando Ngrok...")
        !curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null
        !echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list
        !sudo apt update -qq && sudo apt install -y ngrok
        print("🔑 Puede que necesites autenticar Ngrok. Si ves errores, ejecuta el Paso 1 nuevamente para configurar el token.")
    else:
        print("\n🔌 Ngrok ya está instalado.")

# --- Verificar e instalar Playit.gg si es necesario ---
def check_playit():
    return Path("/usr/local/bin/playit").exists()

if tunnel_type == "playit":
    if not check_playit():
        print("\n🎮 Instalando Playit.gg...")
        !curl -sSL https://github.com/playit-cloud/playit-agent/releases/latest/download/playit-linux-amd64 -o playit
        !chmod +x playit && sudo mv playit /usr/local/bin/
        (server_path / "playit-config").mkdir(exist_ok=True)
        !ln -sf "$(pwd)/playit-config" "/root/.config/playit"
    else:
        print("\n🎮 Playit.gg ya está instalado.")

# --- Verificar que exista el jar ---
server_jar = server_path / "server.jar"
if not server_jar.exists():
    raise FileNotFoundError("❌ No se encontró server.jar. Ejecuta primero el Paso 1 - Configuración Inicial.")

# --- Cambiar a carpeta del servidor ---
os.chdir(server_path)

# --- Detener cualquier servidor Minecraft existente ---
print("🛑 Deteniendo cualquier servidor Minecraft en ejecución...")
!pkill -f "java.*server.jar" || echo "No se encontraron procesos de Minecraft para detener."
time.sleep(5)

# --- Iniciar servidor Minecraft ---
print(f"\n⚡ Iniciando Minecraft {version} con {ram} RAM...")
!nohup java -Xmx{ram} -Xms{ram} -jar "server.jar" --nogui > "server.log" 2>&1 &
time.sleep(10) # Dar tiempo para que el servidor empiece

# --- Iniciar túnel ---
if tunnel_type == "playit":
    print("\n⏳ Iniciando playit...")
    !nohup playit > "playit.log" 2>&1 &
    time.sleep(5)
    print("\n🔗 Posible URL de conexión (playit):")
    !tail -n 10 "playit.log" | grep -o 'https://[^ ]*' || echo "Recarga la página si no aparece la URL"
elif tunnel_type == "ngrok":
    print("\n🔌 Iniciando túnel Ngrok (TCP 25565)...")
    !nohup ngrok tcp 25565 > ngrok.log 2>&1 &
    time.sleep(5)
    def get_ngrok_url():
        try:
            response = requests.get("http://localhost:4040/api/tunnels")
            response.raise_for_status()
            tunnels = response.json().get('tunnels')
            if tunnels:
                return tunnels[0]['public_url'].replace('tcp://', '')
        except requests.exceptions.ConnectionError:
            print("⚠️ Ngrok aún no está listo para la API.")
        except requests.exceptions.RequestException as e:
            print(f"⚠️ Error al obtener la URL de Ngrok: {e}")
        return None

    ngrok_url = get_ngrok_url()
    if ngrok_url:
        print(f"\n🔗 URL de conexión (Ngrok): {ngrok_url}")
    else:
        print("⚠️ No se pudo obtener la URL de Ngrok.")

# --- Mostrar logs ---
print("\n📜 Logs del servidor (Ctrl+C para detener):")
!tail -F "server.log"

In [ ]:
# %% [markdown]
## 🔄 **Paso 3: Reinicio del Servidor (Solo Minecraft)**

# @title Paso 3: Reinicio del Servidor
from pathlib import Path
import os
import subprocess
import time
import requests

# --- Configuración Interactiva ---
tunnel_type = "playit" #@param ["ngrok", "playit"]

# --- Inicialización ---
print("🔄 Reiniciando el servidor de Minecraft...")
server_path = Path("/content/drive/My Drive/Minecraft-Server")
server_jar = server_path / "server.jar"
config_file = server_path / "server_config.txt"
config = {}
try:
    with open(config_file, "r") as f:
        for line in f:
            if "=" in line:
                key, value = line.strip().split("=", 1)
                config[key] = value
    ram = config.get("ram", "6G")
except FileNotFoundError:
    print("⚠️ No se encontró el archivo de configuración. Asegúrate de haber ejecutado el Paso 1.")
    exit()

# --- Verificar archivo del servidor ---
if not server_jar.exists():
    print("❌ No se encontró 'server.jar'. Asegúrate de haber ejecutado el Paso 1.")
    exit()

# --- Cambiar al directorio del servidor ---
os.chdir(server_path)

# --- 1. Detener el servidor Minecraft ---
print("\n🛑 Deteniendo servidor Minecraft...")
!pkill -f "java.*server.jar"
time.sleep(5)  # Esperar a que el servidor se detenga

# --- 2. Reiniciar el servidor Minecraft ---
print(f"\n⚡ Reiniciando servidor Minecraft con {ram} RAM...")
start_command = f"nohup java -Xmx{ram} -Xms{ram} -jar \"{server_jar}\" --nogui > server.log 2>&1 &"
subprocess.Popen(start_command, shell=True)
print("⏳ El servidor de Minecraft se está reiniciando en segundo plano.")
time.sleep(10) # Dar tiempo para que el servidor empiece

# --- 3. Mostrar información de conexión según el túnel ---
print("\n✅ Servidor Minecraft reiniciado.")
if tunnel_type == "ngrok":
    print("🔗 Usando la misma URL de Ngrok de antes.")
    def get_ngrok_url():
        try:
            response = requests.get("http://localhost:4040/api/tunnels")
            response.raise_for_status()
            tunnels = response.json().get('tunnels')
            if tunnels:
                public_url = tunnels[0]['public_url'].replace('tcp://', '')
                return public_url
        except requests.exceptions.ConnectionError:
            print("⚠️ Ngrok aún no está listo para la API.")
        except requests.exceptions.RequestException as e:
            print(f"⚠️ Error al obtener la URL de Ngrok: {e}")
        return None

    ngrok_url = get_ngrok_url()
    if ngrok_url:
        print(f"📍 Posible URL de conexión (Ngrok): {ngrok_url}")
    else:
        print("⚠️ No se pudo obtener la URL de Ngrok en este momento.")

elif tunnel_type == "playit":
    print("🎮 Usando la misma conexión de Playit.gg de antes.")
    print("🔗 Deberías poder conectarte con la misma URL de Playit.gg.")
    !grep -o 'tcp://[^ ]*' <(tail -n 20 playit.log) || echo "Puede que la URL de Playit.gg tarde en aparecer."

# --- 4. Monitorear logs ---
def monitor_logs():
    """Función para monitorear logs en tiempo real"""
    print("\n📜 Iniciando monitor de logs del servidor (Ctrl+C para detener):")
    try:
        with subprocess.Popen(['tail', '-f', 'server.log'],
                              stdout=subprocess.PIPE,
                              stderr=subprocess.PIPE,
                              text=True) as log_process:
            while True:
                line = log_process.stdout.readline()
                if line:
                    print(line.strip())
                time.sleep(0.1)
    except KeyboardInterrupt:
        print("\n🛑 Monitor de logs detenido.")
    except Exception as e:
        print(f"\n⚠️ Error en el monitor de logs: {e}")

monitor_logs()

print("\n💡 El servidor se está ejecutando. Puedes detener el monitor de logs con Ctrl+C.")